In [27]:
import requests, json, time

In [16]:
ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36"

headers = {'User-agent': ua}
cookies = json.load(open('cookies.json', 'r', encoding='utf-8'))

In [26]:
urls = [f'http://ninapro.hevs.ch/system/files/DB6_Preproc/DB6_s{s}_{p}.zip' for s in range(1, 11) for p in {'a', 'b'}]        

In [47]:
response.headers

{'Date': 'Sun, 25 Apr 2021 16:25:46 GMT', 'Server': 'Apache/2.2.3 (Linux/SUSE)', 'X-Powered-By': 'PHP/5.2.14', 'Expires': 'Sun, 19 Nov 1978 05:00:00 GMT', 'Cache-Control': 'private', 'X-Content-Type-Options': 'nosniff', 'Content-Length': '1279928226', 'Keep-Alive': 'timeout=15, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'application/zip'}

In [29]:
#response = requests.head('http://ninapro.hevs.ch/system/files/DB6_Preproc/DB6_s1_a.zip', headers=headers, cookies=cookies)
#int(response.headers['Content-Length']) / (2 ** 30)

def get_size(url):
    response = requests.head(url, headers=headers, cookies=cookies)
    return int(response.headers['Content-Length'])

sizes = [get_size(url) for url in urls]

In [34]:
json.dumps(sizes)

'[1279928226, 846111390, 1120941871, 896722783, 1302960318, 853693771, 1297775784, 872206270, 1284514168, 883103056, 1295413568, 854089005, 1323072356, 882063293, 1377367801, 887222633, 1367504821, 937468118, 1340967542, 895529787]'

In [45]:
total_size_GiB = sum(sizes) / (2 ** 30)
print(f'Dataset size: {total_size_GiB:.1f} GiB')

Dataset size: 20.3 GiB


In [59]:
import json, os, re, requests

ua = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36"

headers = {'User-agent': ua}
cookies = json.load(open('cookies.json', 'r', encoding='utf-8'))

def download_file(subject, part, download_dir='.'):
    filename = f'DB6_s{subject}_{part}.zip'
    url = f'http://ninapro.hevs.ch/system/files/DB6_Preproc/{filename}'
    
    os.makedirs(download_dir, exist_ok=True)
    
    # https://stackoverflow.com/a/1094933
    def sizeof_fmt(num, suffix='B'):
        for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
            if abs(num) < 1024.0:
                return "%3.1f%s%s" % (num, unit, suffix)
            num /= 1024.0
        return "%.1f%s%s" % (num, 'Yi', suffix)
    
    with requests.get(url, headers=headers, cookies=cookies, stream=True) as r:
        r.raise_for_status()
        
        total_filesize_GiB = int(r.headers['Content-Length']) / (2 ** 30)
        print(f'File size: {total_filesize_GiB:.1f} GiB', flush=True)
        
        download_path = os.path.join(download_dir, filename)
        with open(download_path + '.part', 'wb') as f:
            tot_bytes_downloaded = 0
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
                tot_bytes_downloaded += len(chunk)
                print("Downloaded " + sizeof_fmt(tot_bytes_downloaded) + " " * 20 +"\r", end="", flush=True)
    os.rename(download_path + '.part', download_path)
    print("Downloaded in ", download_path, flush=True)

In [60]:
download_file(subject=8, part='a')

File size: 1.3 GiB


KeyboardInterrupt: 